### trade based simulation

# data import using yfinance

## 라이브러리 임포트

In [194]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [195]:
!pip install yfinance

In [196]:
import pandas as pd
import numpy as np
from datetime import datetime
import math
import yfinance as yf

In [197]:
current_date = datetime.now().strftime("%Y-%m-%d-%H-%M-%S")

## 데이터 임포트

In [258]:
tickers = ["SPY", "QQQ", "VTWO"]

In [259]:
divs = []
for ticker in tickers:
  div = yf.Ticker(ticker).dividends
  div.index = pd.to_datetime(div.index).strftime('%Y-%m')
  if not div.index.is_unique:
    div = div.groupby(div.index).sum()
  divs.append(div)

In [260]:
dividends_df = pd.concat(divs, axis=1)
dividends_df.columns = [["Dividend"] * len(tickers), tickers]
dividends_df.index.name = "Date"
dividends_df.sort_index(inplace=True)

In [261]:
tickers_df = yf.download(" ".join(tickers), period='max', interval='1mo')

[*********************100%***********************]  3 of 3 completed


In [262]:
tickers_df.index = pd.to_datetime(tickers_df.index).strftime('%Y-%m')

In [263]:
tickers_df.drop(columns=["Adj Close", "High", "Low", "Open", "Volume"], inplace=True)
tickers_df.dropna(how="all", inplace=True)

In [264]:
tickers_df = tickers_df.reindex(columns=tickers, level=1)

In [265]:
tickers_df = pd.concat([tickers_df, dividends_df], axis=1)

In [268]:
tickers_df.columns = [["Close"] * len(tickers) + ["Dividend"] * len(tickers), tickers * 2]

# backtesting

In [269]:
tickers_df

Close                        Dividend              
                SPY         QQQ       VTWO      SPY    QQQ   VTWO
1993-02   44.406250         NaN        NaN      NaN    NaN    NaN
1993-03   45.187500         NaN        NaN    0.213    NaN    NaN
1993-04   44.031250         NaN        NaN      NaN    NaN    NaN
1993-05   45.218750         NaN        NaN      NaN    NaN    NaN
1993-06   45.062500         NaN        NaN    0.318    NaN    NaN
...             ...         ...        ...      ...    ...    ...
2021-08  451.559998  379.950012  91.199997      NaN    NaN    NaN
2021-09  429.140015  357.959991  88.410004    1.428  0.414  0.221
2021-10  459.250000  386.109985  92.190002      NaN    NaN    NaN
2021-11  455.559998  393.820007  88.360001      NaN    NaN    NaN
2021-12  474.959991  397.850006  89.809998    1.636  0.491  0.475

[347 rows x 6 columns]